# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [29]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests
import json

# Import API key
from api_keys import geoapify_key

In [30]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,ojai,34.4481,-119.2429,20.72,83,21,3.68,US,1696012716
1,1,bredasdorp,-34.5322,20.0403,14.80,87,100,2.57,ZA,1696012717
2,2,afaahiti,-17.7500,-149.2833,25.93,84,45,8.86,PF,1696012717
3,3,salinopolis,-0.6136,-47.3561,29.24,66,28,5.74,BR,1696012717
4,4,port macquarie,-31.4333,152.9167,11.05,89,1,1.53,AU,1696012718


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [52]:
%%capture --no-display

# Configure the map plot
humidity_map = city_data_df.hvplot.points(
    "Lng",
    "Lat", 
    geo = True,
    tiles = "OSM",
    size = "Humidity",
    color = "City",
    title = "Humidity Across the World"
)

# Display the map
humidity_map

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [59]:
# Narrow down cities that fit criteria and drop any results with null values
# limit to places with humidity<50%
low_humidity_cities_df= city_data_df.loc[(city_data_df["Humidity"]< 50) & (city_data_df["Wind Speed"] < 2), :]

# Drop any rows with null values
reduced_low_humidity_cities_df = low_humidity_cities_df.dropna()

# Display sample data
reduced_low_humidity_cities_df

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
30,30,pauini,-7.7136,-66.9764,34.18,45,97,0.30,BR,1696012728
69,69,faya-largeau,17.9257,19.1043,32.15,39,98,1.37,TD,1696012765
71,71,bouarfa,32.5300,-1.9500,23.55,18,5,1.27,MA,1696012765
91,91,albesti,47.7000,27.0667,19.32,37,0,1.98,RO,1696012773
180,180,el porvenir,13.7617,-87.3458,30.63,49,84,1.15,HN,1696012872
196,196,alice springs,-23.7000,133.8833,13.75,38,40,0.00,AU,1696012839
216,216,ghat,24.9647,10.1728,30.36,21,60,1.28,LY,1696012886
233,233,diamantino,-14.4086,-56.4461,34.13,39,95,1.67,BR,1696012914
244,244,markaz-e hukumat-e darweshan,31.1323,64.1934,19.23,18,0,1.97,AF,1696013025
314,314,siyabuswa,-25.1132,29.0445,23.66,34,0,1.64,ZA,1696013097


### Step 3: Create a new DataFrame called `hotel_df`.

In [60]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = reduced_low_humidity_cities_df.copy()
hotel_df = hotel_df[["City", "Country", "Lat", "Lng", "Humidity"]]

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_name = "Hotel Name"
hotel_df["Hotel Name"] = ""

# Display sample data
hotel_df

,City,Country,Lat,Lng,Humidity,Hotel Name
30,pauini,BR,-7.7136,-66.9764,45,
69,faya-largeau,TD,17.9257,19.1043,39,
71,bouarfa,MA,32.5300,-1.9500,18,
91,albesti,RO,47.7000,27.0667,37,
180,el porvenir,HN,13.7617,-87.3458,49,
196,alice springs,AU,-23.7000,133.8833,38,
216,ghat,LY,24.9647,10.1728,21,
233,diamantino,BR,-14.4086,-56.4461,39,
244,markaz-e hukumat-e darweshan,AF,31.1323,64.1934,18,
314,siyabuswa,ZA,-25.1132,29.0445,34,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [61]:
# Set parameters to search for a hotel
categories = "accommodation.hotel"
radius = 10000
limit = 20

    #setup parameters dictionary:
params = {
    "categories": categories,
    "limit": limit,
    "apiKey": geoapify_key
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    latitude = row["Lat"]
    longitude = row["Lng"]
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{longitude},{latitude},{radius}"
    params["bias"] = f"proximity:{longitude},{latitude}"
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"

    # Make and API request using the params dictionaty
    name_address = requests.get(base_url, params=params)
    
    # Convert the API response to JSON format
    name_address = name_address.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
pauini - nearest hotel: No hotel found
faya-largeau - nearest hotel: No hotel found
bouarfa - nearest hotel: Climat du Maroc مناخ المغرب
albesti - nearest hotel: No hotel found
el porvenir - nearest hotel: No hotel found
alice springs - nearest hotel: Aurora Alice Springs
ghat - nearest hotel: فندق تاسيلي
diamantino - nearest hotel: Hotel kayaby
markaz-e hukumat-e darweshan - nearest hotel: No hotel found
siyabuswa - nearest hotel: No hotel found
zaysan - nearest hotel: Рахат қонақүй
gevgelija - nearest hotel: Apollonia Hotel
campbellton - nearest hotel: Campbellton City Center Mall
melenci - nearest hotel: No hotel found
divrigi - nearest hotel: No hotel found
shihezi - nearest hotel: 石河子宾馆
zhangye - nearest hotel: Zhangye Hotel 张掖饭店
puerto carreno - nearest hotel: Carreño Plaza
wad rawah - nearest hotel: No hotel found
lagunas - nearest hotel: Hospedaje Eco
newman - nearest hotel: No hotel found
quchan - nearest hotel: هتل خیام
turpan - nearest hotel: Oriental
l

,City,Country,Lat,Lng,Humidity,Hotel Name
30,pauini,BR,-7.7136,-66.9764,45,No hotel found
69,faya-largeau,TD,17.9257,19.1043,39,No hotel found
71,bouarfa,MA,32.5300,-1.9500,18,Climat du Maroc مناخ المغرب
91,albesti,RO,47.7000,27.0667,37,No hotel found
180,el porvenir,HN,13.7617,-87.3458,49,No hotel found
196,alice springs,AU,-23.7000,133.8833,38,Aurora Alice Springs
216,ghat,LY,24.9647,10.1728,21,فندق تاسيلي
233,diamantino,BR,-14.4086,-56.4461,39,Hotel kayaby
244,markaz-e hukumat-e darweshan,AF,31.1323,64.1934,18,No hotel found
314,siyabuswa,ZA,-25.1132,29.0445,34,No hotel found


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [62]:
%%capture --no-display

# Configure the map plot
hotel_map = hotel_df.hvplot.points(
    'Lng', 
    'Lat', 
    geo=True, 
    tiles="OSM",
    color='City',
    hover_cols= ['Humidity', 'Hotel Name', 'Country'],
    title= 'Potential Hotels for My Ideal Vacation'
)

# Display the map
hotel_map

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Hotel Name,Country)